# Math 753/853 HW2: Mathematics in finite precision

## Before you begin!

**First rename this file "math753-hw2-mylastname"** using your last name in the indicated spot. With the file extension, the full filename should be "math753-hw2-mylastname.ipynb". No capital letters, please. 

**To submit the homework** email it to `john.gibson@unh.edu` with the subject "MATH 753 HW2" by midnight, Sunday Oct 1, 2017.

**Writing explanations in your solutions.**  Many of the problems ask for *explanations* of answers and calculations. For these you should write one to several complete sentences that explain the reasoning behind your work in a manner that would be helpful to a fellow student.

## Problem 0: example with solution

Use the Conditioning and Accuracy Theorem to estimate a value for the expected relative error $|\tilde{f} - f|\,/\,|f|$ for the floating-point calculation $\tilde{f}(x)$ of the mathematical problem $f(x)$ in 64-bit arithmetic. Then devise a calculation in Julia that confirms your expectations. Explain each of your answers.

**Subtraction, very different numbers.** Estimate a value for the relative error for floating-point calculation of subtraction, $f(x_1, x_2) = x_2 - x_1$, using $x_1 = 17/13 \times 10^{23}$ and $x_2 = 19/57$. Confirm your estimate by calculating the relative error in Julia. Explain each answer.

In [2]:
# The condition number of positive floating-point subtraction f(x₁, x₂) = x₂ - x₁ is
# κ = max(x₁, x₂)/|x₂ - x₁| 
# 
# for x₁ = 17/13 × 10²³ and x₂ = 19/57, this works out to 
#
# κ(x) = 17/13 × 10²³ / |17/13 × 10²³ - 19/57| ≈ 1
#
# The conditioning and accuracy theorem then estimates the relative error 
# of the 64-bit calculation to be

# |f̃ - f|/|f| = O(κ(x) ϵₘ) ≈ 2e-16

# Let's verify that with a direct calculation in Julia. Use BigFloats to represent the
# "real" numbers in high precision and compute the "true" value f, and then calculate
# the floating-point value f̃ in Float64. 

x₁ = BigFloat(17//13) * BigFloat(10)^23
x₂ = BigFloat(19//57)

f = x₂ - x₁

f̃ = Float64(x₂) - Float64(x₁)

abs(f̃ - f)/abs(f)

3.454132960784313725490204883084017685505574778953623547496062039450477003140082e-17

The relative error is about 3e-17, which slightly smaller than the estimate with machine precision 2e-16. 

## Problem 1

What is the next `Float64` number after 6.0? Determine the precise value based on the structure of the 64-bit floating point number system as presented in lecture. Explain your reasoning. Then confirm your answer with a few calculations in Julia, and explain how the calculations confirm your expectations. 

In [5]:
# In Float64, the next number after 1 is 1 + 2^-52, since there are 52 bits allocated to the mantissa. 
# So for numbers between 1 and 2, the spacing is 2^-52. Larger numbers are built by multiplying these by powers of 2.
# So for numbers between 2 and 4, the spacing is 2 × 2^-52 == 2^-51
# So for numbers between 4 and 8, the spacing is 4 × 2^-52 == 2^-50. Let's try that out

6.0 + 2.0^-50

6.000000000000001

In [6]:
(6.0 + 2.0^-50) - 6.0

8.881784197001252e-16

In [7]:
2.0^-50

8.881784197001252e-16

In [8]:
# Yep, we can definitely represent 6 + 2^-50. Can we fit another number between 6 and that? 
# Let's try
6.0 + 2.0^-51

6.0

In [9]:
# Doesn't look like it. Try subtracting off 6 to see exactly what the difference is
(6.0 + 2.0^-51) - 6.0

0.0

In [10]:
# Ok, that shows that 6.0 + 2.0^-51 rounds down to 6.0 exactly.
# Julia actually has a convenient function for finding the next float after a given number
nextfloat(6.0)

6.000000000000001

In [11]:
# and we can see the difference by subtracting 
nextfloat(6.0) - 6.0

8.881784197001252e-16

In [12]:
# Again, that is 2^-50. So the next Float64 after 6 is 6 + 2^-50

## Problem 2

What range of integers can be represented exactly in the `Float64` number system? I.e. what is the maximum value of $N$ for which all integers between $-N$ and $N$ are represented exactly as `Float64`s? As before, determine the answer based on the structure of the 64-bit floating-point number system as presented in lecture, and explain your reasoning. Then confirm your answer with a few calculations in Julia, and explain how the calculations confirm your expectations. 

In [13]:
# In lecture we saw that Float 64s had form
#
# sign * (1 + b1 2^-1 + b2 2^-2 + ... + b52 2^-52) * 2^e
# 
# Transfer 2^n from 2^e onto the sum to get
#
# sign * (2^n + b1 2^(n-1) + b2 2^(n-2) + ... + b52 2^(n-52)) 2^(e-n)
#
# So you can produce any integer up to 2^53 - 1 by figuring out its binary expansion
# and encoding it in the above form, where 2^n is the leading value in the binary expansion
# and e=n so that 2^(e-n) = 1
#
#     binary     
# 1 =   1    =   2^0
# 2 =  10    =   2^1
# 3 =  11    =   2^1 + 2^0
# 4 = 100    =   2^2 +  0 
# 5 = 101    =   2^2 +  0  + 2^0
# 6 = 110    =   2^2 + 2^1 +  0 
# 7 = 111    =   2^2 + 2^1 + 2^0
#
# etc. That works up to n=52
#
# 2^53 - 1   =   2^52 + 2^51 + 2^50 + ... + 2^2 + 2^1 + 2^0
#
# After that, we can only get larger numbers by increasing the exponent, i.e. multiplying 
# previous numbers by powers of two. That means no further odd numbers can be represented
# exactly as Float64s! We can represent 2^53 exactly but not 2^53 + 1. So the range of
# integers that can be represented exactly as Float64's is -2^53 to 2^53. Let's verify.

2.0^53

9.007199254740992e15

In [5]:
# show that 2.0^53 is exactly the integer 2^53
2^53 

9007199254740992

In [ ]:
# The decimal number produced by the print(Float64) function
# produces the same digits 9007199254740992 as the integer
# calculation 2^53, but that's not a perfect assurance that
# the Float64 value of 2.0^53 is exactly the integer 2^53
# Let's repeat casting to BigFloat, which should reveal any
# very small differences

In [7]:
BigFloat(2.0^53) - 2^53

0.000000000000000000000000000000000000000000000000000000000000000000000000000000

In [ ]:
# The difference is zero to the resolution of BigFloat. 
# I'll take that as assurance that 2.0^53 in Float64 is exactly 2^53

In [8]:
# show that 2.0^53 - 1.0 is exactly the integer 2^53 - 1
BigFloat(2.0^53 - 1.0)  - (2^53 - 1)

0.000000000000000000000000000000000000000000000000000000000000000000000000000000

In [9]:
# show that 2.0^53 + 1.0 is not the integer 2^53 + 1
BigFloat(2.0^53 + 1.0)  - (2^53 + 1)

-1.000000000000000000000000000000000000000000000000000000000000000000000000000000

In [11]:
# Show all the same for -2^53

# these two numbers should be equal
BigFloat(-2.0^53)  - (-2^53)

0.000000000000000000000000000000000000000000000000000000000000000000000000000000

In [12]:
# these two number should be equal
BigFloat(-2.0^53 + 1.0)  - (-2^53 + 1)

0.000000000000000000000000000000000000000000000000000000000000000000000000000000

In [13]:
# these two numbera should be different
BigFloat(-2.0^53 - 1.0)  - (-2^53 - 1)

1.000000000000000000000000000000000000000000000000000000000000000000000000000000

In [19]:
# This confirms that the range of exact integer Float64s is -2^53 to 2^53

## Problem 3

There is a debate underway in the Julia development community whether the time library should represent time with floating-point numbers or with integers. (https://discourse.julialang.org/t/why-do-time-quantities-have-to-be-integers/5864/51) Suppose you want to measure time with millisecond accuracy near the present, but also dates as far back as 0 BC, using just a single number. What kind of number should you use, integer or floating-point? How many bits would you need? Explain your reasoning. 

In [20]:
# How many milliseconds ago was 0 B.C.? 
# (1000 ms per s) * (60 s/min) * (60 min/hr) * (24 hr/day) * (365.24 day/yr) * (2017 yr)
1000*60*60*24*365.24*2017

6.3649936512e13

In [22]:
# Ok, 0 BC was 6 × 10¹³ milliseconds ago. So if we count in milliseconds with integers, 
# we need to be able to represent integers up to at least 6 × 10¹³ to go back to 0 BC

# Int32 spans 2^-31 to 2^31. Is 2^31 big enough?
2.0^31

2.147483648e9

In [23]:
# 2^31 ≈ 2e09 << 6e13, so Int32 is not big enough to count in milliseconds back to 0 BC!

# Let's try Int64, which spans -2^63 to 2^63
2.0^63

9.223372036854776e18

In [24]:
# 2^63 ≈ 1e19 >> 6e13, so Int64 is big enough to count in milliseconds back to 0 BC
#
# Still, I would prefer to use a floating point number system to represent time, 
# with t=0 somewhere near the present (like the Unix epoch of 12:00am Jan 1 1970)
# since this would allow very high precision time stamps near the present (1e-308 
# seconds, for example) and very long times going into the future and past (±1e300 
# seconds, for example). 

## Problems 4-10

For each of problems 4 through 10, use the Conditioning and Accuracy Theorem to estimate a value for the expected relative error $|\tilde{f} - f|\,/\,|f|$ for the floating-point calculation $\tilde{f}(x)$ of the mathematical problem $f(x)$ in 64-bit arithmetic. Then devise a calculation in Julia that confirms your expectations. Explain each of your answers.

## Problem 4

**Multiplication, order-1 numbers.** Estimate a value for the relative error for floating-point calculation of multiplication $f(x) = cx$, with  $c=7/11$ and $x=19/3$ (numbers both close to 1). Confirm your estimate by calculating the relative error in Julia. Explain each answer.


In [35]:
# The condition number of multiplication f(x) = cx is κ = 1, from lecture notes.
# Therefore we expect a relative error # |f̃ - f|/|f| = O(κ(x) ϵₘ) ≈ 2e-16 using 64-bit floats

c = BigFloat(7//11)
x = BigFloat(19//3)

f = c*x
f̃ = Float64(c)*Float64(x)

@show abs(f̃ - f)/abs(f)

abs(f̃ - f) / abs(f) = 6.678033230828009265706055146410949248120300751879699248120313738599330969089577e-18


6.678033230828009265706055146410949248120300751879699248120313738599330969089577e-18

In [36]:
# As expected, the relative error of 1e-17 is within the estimate O(κ(x) ϵₘ) ≈ 2e-16 

## Problem 5

**Multiplication, large numbers.** Estimate a value for the relative error for floating-point calculation of multiplication, $f(x) = cx$, using $c=2/3 \times 10^{72}$ and $x=5/7 \times 10^{200}$ (two very large numbers). Confirm your estimate by calculating the relative error in Julia. Explain each answer.

In [37]:
# The condition number of multiplication f(x) = cx is κ = 1, from lecture notes.
# It doesn't matter if x or c is big. We still expect a relative error 
# |f̃ - f|/|f| = O(κ(x) ϵₘ) ≈ 2e-16 using 64-bit floats

c = BigFloat(2//3) * BigInt(10)^72   # use BigInt here to avoid integer overflow in 10^72
x = BigFloat(5//7) * BigInt(10)^200  # ditto

f = c*x
f̃ = Float64(c)*Float64(x)

@show abs(f̃ - f)/abs(f)

abs(f̃ - f) / abs(f) = 5.050757729985847810417518427908503871749724466951876820773350005040136410524203e-17


5.050757729985847810417518427908503871749724466951876820773350005040136410524203e-17

In [38]:
# As expected, the relative error of 1e-16 is within the estimate O(κ(x) ϵₘ) ≈ 2e-16 

## Problem 6

**Addition, order-1 numbers.** Estimate a value for the relative error for floating-point calculation of addition, $f(x_1, x_2) = x_1 + x_2$, using $x_1 = 17/3$ and $x_2 = 5/11$ (two numbers near 1). Confirm your estimate by calculating the relative error in Julia. Explain each answer.

In [39]:
# The condition number of addition f(x₁, x₂) = x₁ + x₂ is κ(x) = max(x₁, x₂)/|x₁ + x₂|
# for positive x₁, x₂. For these numbers, κ(x) ≈ 1, so we expect  
# |f̃ - f|/|f| = O(κ(x) ϵₘ) ≈ 2e-16 using 64-bit floats

x₁ = BigFloat(17//3)
x₂ = BigFloat(5//11)


f = x₁ + x₂
f̃ = Float64(x₁) + Float64(x₂)

@show abs(f̃ - f)/abs(f)

abs(f̃ - f) / abs(f) = 1.758769147921040064037436305886448019801980198019801980198019836182845762750271e-17


1.758769147921040064037436305886448019801980198019801980198019836182845762750271e-17

In [40]:
# The relative error 1e-17 is within the estimate O(κ(x) ϵₘ) ≈ 2e-16 

## Problem 7

**Subtraction, very close numbers.** Estimate a value for the relative error for floating-point calculation of subtraction, $f(x_1, x_2) = x_2 - x_1$, using $x_1 = 129/11$ and $x_2 = x_1 + \delta$, where $\delta = 10^{-13}$. Confirm your estimate by calculating the relative error in Julia. Explain each answer.



In [16]:
# The condition number of subtraction f(x₁, x₂) = x₁ - x₂ is κ(x) = max(x₁, x₂)/|x₁ - x₂|
# for positive x₁, x₂. For the given numbers,  κ(x) ≈ (129/11)/10⁻¹³ ≈ 10¹⁴ , so we expect  
# |f̃ - f|/|f| = O(κ(x) ϵₘ) ≈ 2 × 10⁻¹⁶ × 10¹⁴ ≈ 10⁻² using 64-bit floats

x₁ = BigFloat(129//11)
x₂ = x₁ + 1e-13


f = x₁ - x₂
f̃ = Float64(x₁) - Float64(x₂)

abs(f̃ - f)/abs(f)

1.252339845814273411222399954174929545256895052821782426447567000725618395212299e-02

In [17]:
# Yes, indeed, the relative error is about 10⁻².

## Problem 8

**Powers, big $x$, small $n$.** Estimate a value for the relative error for floating-point calculation of the power function $f(x) = x^n$ for $x = 11/7 \times 10^{13}$ and $n=12$. Confirm your estimate by calculating the relative error in Julia. Explain each answer.

In [19]:
# The condition number of the power function f(x) = x^n is κ(x) = n, so we expect  
# |f̃ - f|/|f| = O(κ(x) ϵₘ) ≈ 12 × 10⁻¹⁶  ≈ 10⁻¹⁵ using 64-bit floats

x = BigFloat(11//7)

f = x^12
f̃ = Float64(x)^12

abs(f̃ - f)/abs(f)


3.019177961309076418305554568647032572229231436002898329211994075881676894681556e-16

In [20]:
# The relative error 3e-16 is slightly less than the estimated error 1e-15

## Problem 9

**Powers, small $x$, big $n$.** Estimate a value for the relative error for floating-point calculation of the power function $f(x) = x^n$ for $x = 12$ and $n = 11/7 \times 10^{13}$. Compare your estimate to the relative error calculated in Julia. Explain each answer.

In [23]:
# The condition number of the power function f(x) = x^n is κ(x) = n, so we expect  
# |f̃ - f|/|f| = O(κ(x) ϵₘ) ≈ 10¹³ × 10⁻¹⁶  ≈ 10⁻³ using 64-bit floats

x = BigFloat(12)
n = BigFloat(11//7) * BigFloat(10)^13
f = x^n
f̃ = Float64(x)^Float64(n)

err = abs(f̃ - f)/abs(f)

BigFloat(Inf, 256)

In [25]:
# Whoa.... whats going on here? The error is infinite, not 10⁻³!
@show f
@show f̃


f = 1.767228807095811065535164682326909225491843761406900379067798444373528796590833e+16958562437891
f̃ = Inf


Inf

In [26]:
# The problem is that the exact answer overflows the maximum exponent of Float64, which is about 308 
# in base 10. So that Float64 calculation overflows to Inf and is not accurate at all! Our Conditioning
# and Accuracy Theorem applies to an *idealized* floating-point number system which has no limitations
# on exponents! 

## Problem 10

**Solution of linear system.** Estimate a value for the relative error for numerical solution of the linear system $Ax=b$ for the given $A$ and $b$. Compare your estimate to a direct calculation of the relative error in Julia. Explain.


Note that in this case the data is $A,b$, the solution is $x$, the mathematical function is is $f(A,b) = A^{-1}b = x$, and the relative error is $|\tilde{x} - x\|\, / \, \|x\|$. The condition number of the $Ax=b$ solve is $\kappa(A) = \|A\| \|A^{-1}\|$, which can be calculated in Julia with `cond(A)`. 

In [27]:
function randA(m, κ)
    σ = logspace(0, -log10(κ), m)
    Σ = diagm(σ)
    U,tmp = qr(randn(m,m))
    V,tmp = qr(randn(m,m))
    A = U*Σ*V'
end
A = randA(5, 1e15);      # generate a random matrix A with condition number 1e15
x = randn(5);            # generate a random vector x, the true solution of Ax=b
b = A*x;                 # determine the right-hand-side vector b s.t. Ax=b

In [28]:
cond(A)

9.504669289974245e14

In [29]:
# The linear-system a.k.a. Ax=b problem f: b -> x = A⁻¹ b has condition number κ(b) 
# which is bounded by κ(a) ≈ 10¹⁵. So we expect a realtive error in the solution x
# O(κ(A) ϵₘ) ≈ 10¹⁵ × 10⁻¹⁶ ≈ 0.1. Let's check it out!

x̂ = A\b

norm(x̂-x)/norm(x)

0.015088056736573319

In [31]:
# Yep, the actual error is smaller than but on the order of the estimate 0.1.
# Notice how we used norm instead of abs in the error evaluations, because here
# the solution x is a vector
@show x;
@show x̂;

x = [-0.949067, -0.458087, 2.04242, 0.431833, -0.130941]
x̂ = [-0.959384, -0.481666, 2.05676, 0.446741, -0.11835]


## Happy computing!